# Scenariusze testowe dla porównania wydajności baz danych

### 1. Operacja CREATE

- Dodanie nowego nauczyciela
- Utworzenie nowej klasy
- Dodanie nowego przedmiotu
- Zarejestrowanie nowego ucznia
- Przypisanie ucznia do klasy
- Utworzenie harmonogramu zajęć
- Wystawienie oceny

### 2. Operacja READ

Pobranie kompleksowego raportu zawierającego:
- Dane osobowe ucznia
- Informacje o klasie
- Dane nauczyciela prowadzącego
- Listę ocen z opisami przedmiotów
- Szczegółowy harmonogram zajęć

### 3. Operacja UPDATE

- Aktualizacja danych ucznia
- Zmiana przypisania do klasy
- Modyfikacja nazwy klasy
- Aktualizacja danych nauczyciela
- Zmiana oceny
- Aktualizacja opisu przedmiotu
- Modyfikacja harmonogramu zajęć

### 4. Operacja DELETE

- Usunięcie ocen ucznia
- Wypisanie ucznia z klasy
- Usunięcie harmonogramu zajęć
- Usunięcie klasy
- Opcjonalne usunięcie przedmiotów
- Opcjonalne usunięcie nauczyciela
- Usunięcie rekordu ucznia

## Ilość rekordów do testów

Testy będą przeprowadzane dla następujących ilości rekordów:

1. 10,000 rekordów
2. 100,000 rekordów
3. 1,000,000 rekordów
4. 10,000,000 rekordów

## Metryki wydajnościowe

Dla każdego scenariusza i ilości rekordów będziemy mierzyć:

1. Czas wykonania całego scenariusza
2. Średni czas pojedynczych operacji
3. Liczbę operacji na sekundę (throughput)
4. Zużycie zasobów systemowych (CPU, RAM, I/O dysku)

# Narzędzia i technologie testowe

### Wbudowane instrumenty bazodanowe

Każdy system oferuje specjalizowane narzędzia diagnostyczne:


| System | Narzędzie | Funkcjonalności |
| :-- | :-- | :-- |
| PostgreSQL | pgBench | Testy TPC-B, własne skrypty SQL |
| MariaDB | sysbench | Testy OLTP, skalowanie pionowe |
| MongoDB | mongoperf | Operacje na dokumentach JSON |
| Cassandra | cassandra-stress | Testy dystrybucji danych |
| Redis | redis-benchmark | Pomiar opóźnień operacji klucz-wartość |

Wykorzystanie natywnych narzędzi pozwala na precyzyjne badanie specyficznych mechanizmów storage engine.

### Automatyzacja w Pythonie

Kluczowe biblioteki wspierające testy:

- **SQLAlchemy** dla baz relacyjnych
- **PyMongo** dla MongoDB
- **Cassandra-driver** dla Cassandra
- **redis-py** dla Redis


In [1]:
import psycopg2
from pymongo import MongoClient
from cassandra.cluster import Cluster
import redis
import mysql.connector
import yaml

with open('docker-compose.yml', 'r') as file:
    docker_config = yaml.safe_load(file)

# PostgreSQL connection
postgres_config = docker_config['services']['postgresql']
postgres_client = psycopg2.connect(
    host='localhost',  # or 'postgresql' if running inside Docker network
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTGRES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

# MariaDB connection
mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',  # or 'mariadb' if running inside Docker network
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0]
)

# Cassandra connection
cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'],  # or ['cassandra'] if running inside Docker network
                        port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

# MongoDB connection
mongo_config = docker_config['services']['mongodb']
mongo_client = MongoClient(
    host='localhost',  # or 'mongodb' if running inside Docker network
    port=int(mongo_config['ports'][0].split(':')[0])
)

# Redis connection
redis_config = docker_config['services']['redis']
redis_client = redis.Redis(
    host='localhost',  # or 'redis' if running inside Docker network
    port=int(redis_config['ports'][0].split(':')[0])
)

In [2]:
# Simple sanity checks for each database connection

# PostgreSQL check
try:
    with postgres_client.cursor() as cursor:
        cursor.execute("SELECT 1")
        print("PostgreSQL connection successful")
except Exception as e:
    print(f"PostgreSQL connection failed: {e}")

# MariaDB check
try:
    with mariadb_client.cursor(buffered=True) as cursor:
        cursor.execute("SELECT 1")
        print("MariaDB connection successful")
except Exception as e:
    print(f"MariaDB connection failed: {e}")


# Cassandra check
try:
    cassandra_session.execute("SELECT release_version FROM system.local")
    print("Cassandra connection successful")
except Exception as e:
    print(f"Cassandra connection failed: {e}")

# MongoDB check
try:
    mongo_client.admin.command('ping')
    print("MongoDB connection successful")
except Exception as e:
    print(f"MongoDB connection failed: {e}")

# Redis check
try:
    redis_client.ping()
    print("Redis connection successful")
except Exception as e:
    print(f"Redis connection failed: {e}")


PostgreSQL connection successful
MariaDB connection successful
Cassandra connection successful
MongoDB connection successful
Redis connection successful


In [3]:
!pip install pandas

In [4]:
!pip install sqlalchemy

### Import danych - PostgreSQL

In [5]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Konfiguracja połączenia z PostgreSQL
DB_URI = (
    f"postgresql://{postgres_config['environment']['POSTGRES_USER']}:"
    f"{postgres_config['environment']['POSTGRES_PASSWORD']}@"
    f"localhost:"  # Możesz zmienić na 'postgresql', jeśli działa w sieci Dockera
    f"{postgres_config['ports'][0].split(':')[0]}/"
    f"{postgres_config['environment']['POSTGRES_DB']}"
)
engine = create_engine(DB_URI)

# Ścieżka do folderu z plikami CSV
folder_path = "10tys"

# Pobranie listy wszystkich plików CSV w folderze
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Iteracja po plikach CSV
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    table_name = os.path.splitext(file)[0]  # Nazwa tabeli na podstawie nazwy pliku
    
    try:
        # Wczytanie danych z pliku CSV do DataFrame
        df = pd.read_csv(file_path)
        
        # Import danych do bazy danych (tworzenie tabeli automatycznie)
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        
        print(f"Pomyślnie zaimportowano dane z {file} do tabeli {table_name}.")
    except Exception as e:
        print(f"Błąd podczas importu pliku {file}: {e}")


Pomyślnie zaimportowano dane z classes.csv do tabeli classes.
Pomyślnie zaimportowano dane z enrollments.csv do tabeli enrollments.
Pomyślnie zaimportowano dane z grades.csv do tabeli grades.
Pomyślnie zaimportowano dane z schedules.csv do tabeli schedules.
Pomyślnie zaimportowano dane z students.csv do tabeli students.
Pomyślnie zaimportowano dane z subjects.csv do tabeli subjects.
Pomyślnie zaimportowano dane z teachers.csv do tabeli teachers.


In [6]:
#Sprawdzenie działania
try:
    with postgres_client.cursor() as cursor:
        cursor.execute("SELECT * FROM classes;")
        result = cursor.fetchall()
        for row in result: 
            print(row)
except Exception as e:
    print(f"PostgreSQL failed: {e}")


(1, 'Appear', 9900, '2024-11-21 18:00:21')
(2, 'Buy', 5801, '2021-11-12 03:14:49')
(3, 'Community', 8692, '2024-04-09 21:47:12')
(4, 'Ability', 2994, '2020-03-12 12:14:32')
(5, 'There', 8880, '2023-12-16 10:30:15')
(6, 'Station', 266, '2020-10-18 08:07:36')
(7, 'Day', 5288, '2024-02-11 23:57:14')
(8, 'Something', 835, '2022-02-19 08:43:46')
(9, 'Yard', 2464, '2022-11-20 23:48:21')
(10, 'General', 2618, '2022-04-03 07:44:41')
(11, 'Interesting', 8396, '2023-12-03 13:44:08')
(12, 'Old', 4117, '2023-12-30 11:01:15')
(13, 'Maintain', 1048, '2023-11-02 12:50:35')
(14, 'Blood', 1624, '2020-06-28 03:50:56')
(15, 'Buy', 7461, '2023-08-10 07:49:16')
(16, 'Nor', 1865, '2020-06-12 15:37:24')
(17, 'Number', 5707, '2021-08-12 11:24:17')
(18, 'Election', 832, '2024-12-19 08:58:27')
(19, 'Understand', 2233, '2024-06-19 17:14:41')
(20, 'Indicate', 8497, '2020-02-11 23:59:40')
(21, 'These', 6659, '2020-01-06 05:58:33')
(22, 'Less', 992, '2024-10-20 03:03:18')
(23, 'Market', 8956, '2024-04-01 05:09:50')

### Import danych - MariaDB

In [7]:
!pip install pymysql

In [8]:
# Konfiguracja połączenia z MariaDB na podstawie docker_config

DB_URI = (
    f"mysql+pymysql://{docker_config['services']['mariadb']['environment']['MYSQL_USER']}:"
    f"{docker_config['services']['mariadb']['environment']['MYSQL_PASSWORD']}@"
    f"localhost:"  # lub 'mariadb' dla połączenia wewnątrz Dockera
    f"{docker_config['services']['mariadb']['ports'][0].split(':')[0]}/"
    f"{docker_config['services']['mariadb']['environment']['MYSQL_DATABASE']}"
)
engine = create_engine(DB_URI)

# Ścieżka do folderu z plikami CSV
#folder_path = "10tys"

# Pobranie listy wszystkich plików CSV w folderze
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Iteracja po plikach CSV
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    table_name = os.path.splitext(file)[0]  # Nazwa tabeli na podstawie nazwy pliku
    
    try:
        # Wczytanie danych z pliku CSV do DataFrame
        df = pd.read_csv(file_path)
        
        # Import danych do bazy danych (tworzenie tabeli automatycznie)
        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='replace',  # 'append' dla dodawania danych
            index=False
        )
        
        print(f"Pomyślnie zaimportowano dane z {file} do tabeli {table_name}.")
    except Exception as e:
        print(f"Błąd podczas importu pliku {file}: {e}")




Pomyślnie zaimportowano dane z classes.csv do tabeli classes.
Pomyślnie zaimportowano dane z enrollments.csv do tabeli enrollments.
Pomyślnie zaimportowano dane z grades.csv do tabeli grades.
Pomyślnie zaimportowano dane z schedules.csv do tabeli schedules.
Pomyślnie zaimportowano dane z students.csv do tabeli students.
Pomyślnie zaimportowano dane z subjects.csv do tabeli subjects.
Pomyślnie zaimportowano dane z teachers.csv do tabeli teachers.


In [9]:
# Sprawdzenie działania
try:
    with mariadb_client.cursor() as cursor:
        cursor.execute("SELECT * FROM classes;")
        result = cursor.fetchall()
        for row in result: 
            print(row)
except Exception as e:
    print(f"PostgreSQL failed: {e}")

(1, 'Appear', 9900, '2024-11-21 18:00:21')
(2, 'Buy', 5801, '2021-11-12 03:14:49')
(3, 'Community', 8692, '2024-04-09 21:47:12')
(4, 'Ability', 2994, '2020-03-12 12:14:32')
(5, 'There', 8880, '2023-12-16 10:30:15')
(6, 'Station', 266, '2020-10-18 08:07:36')
(7, 'Day', 5288, '2024-02-11 23:57:14')
(8, 'Something', 835, '2022-02-19 08:43:46')
(9, 'Yard', 2464, '2022-11-20 23:48:21')
(10, 'General', 2618, '2022-04-03 07:44:41')
(11, 'Interesting', 8396, '2023-12-03 13:44:08')
(12, 'Old', 4117, '2023-12-30 11:01:15')
(13, 'Maintain', 1048, '2023-11-02 12:50:35')
(14, 'Blood', 1624, '2020-06-28 03:50:56')
(15, 'Buy', 7461, '2023-08-10 07:49:16')
(16, 'Nor', 1865, '2020-06-12 15:37:24')
(17, 'Number', 5707, '2021-08-12 11:24:17')
(18, 'Election', 832, '2024-12-19 08:58:27')
(19, 'Understand', 2233, '2024-06-19 17:14:41')
(20, 'Indicate', 8497, '2020-02-11 23:59:40')
(21, 'These', 6659, '2020-01-06 05:58:33')
(22, 'Less', 992, '2024-10-20 03:03:18')
(23, 'Market', 8956, '2024-04-01 05:09:50')

In [10]:
from sqlalchemy import create_engine, Column, Integer, String, Date, DateTime, ForeignKey, MetaData
from sqlalchemy.orm import declarative_base, sessionmaker
import os
import pandas as pd

Base = declarative_base()

# Definicja modeli
class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    birth_date = Column(Date)
    created_at = Column(DateTime)

class Teacher(Base):
    __tablename__ = 'teachers'
    id = Column(Integer, primary_key=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    subject = Column(String(100))
    hire_date = Column(Date)
    created_at = Column(DateTime)

class Class(Base):
    __tablename__ = 'classes'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    teacher_id = Column(Integer, ForeignKey('teachers.id'))
    created_at = Column(DateTime)

class Subject(Base):
    __tablename__ = 'subjects'
    id = Column(Integer, primary_key=True)
    name = Column(String(100))
    description = Column(String)
    created_at = Column(DateTime)

class Grade(Base):
    __tablename__ = 'grades'
    id = Column(Integer, primary_key=True)
    student_id = Column(Integer, ForeignKey('students.id'))
    subject_id = Column(Integer, ForeignKey('subjects.id'))
    grade = Column(Integer)
    created_at = Column(DateTime)

class Enrollment(Base):
    __tablename__ = 'enrollments'
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    class_id = Column(Integer, ForeignKey('classes.id'), primary_key=True)
    enrolled_at = Column(DateTime)

class Schedule(Base):
    __tablename__ = 'schedules'
    id = Column(Integer, primary_key=True)
    class_id = Column(Integer, ForeignKey('classes.id'))
    subject_id = Column(Integer, ForeignKey('subjects.id'))
    day_of_week = Column(String(10))
    time_start = Column(DateTime)
    time_end = Column(DateTime)

def drop_all_tables(connection_uri):
    engine = create_engine(connection_uri)
    meta = MetaData()
    meta.reflect(bind=engine)
    meta.drop_all(bind=engine)
    print("All tables dropped.")

def setup_database(connection_uri):
    engine = create_engine(connection_uri)
    Base.metadata.create_all(engine)
    return sessionmaker(bind=engine)  # Zwracamy fabrykę sesji, a nie konkretną sesję

def import_data(SessionFactory, csv_folder):
    with SessionFactory() as session:
        try:
            model_mapping = {
                'students': Student,
                'teachers': Teacher,
                'classes': Class,
                'subjects': Subject,
                'grades': Grade,
                'enrollments': Enrollment,
                'schedules': Schedule
            }

            for file in os.listdir(csv_folder):
                if file.endswith('.csv'):
                    table_name = os.path.splitext(file)[0]
                    model = model_mapping.get(table_name)

                    if not model:
                        continue

                    file_path = os.path.join(csv_folder, file)

                    # Wczytaj dane z kontrolą typów
                    df = pd.read_csv(file_path,skiprows=1, dtype={
                        'teacher_id': 'Int64',
                        'student_id': 'Int64',
                        'class_id': 'Int64',
                        'subject_id': 'Int64',
                        'grade': 'Int64'
                    }).dropna(subset=[
                        'teacher_id',
                        'student_id',
                        'class_id',
                        'subject_id'
                    ], how='any')

                    # Zamień NaN na wartości domyślne
                    df = df.where(pd.notnull(df), None)

                    # Podział na partie
                    chunks = [df[i:i+10000] for i in range(0, df.shape[0], 10000)]

                    for chunk in chunks:
                        # Konwersja typów dla MySQL
                        for col in ['teacher_id', 'student_id', 'class_id', 'subject_id']:
                            if col in chunk.columns:
                                chunk[col] = chunk[col].astype('Int64').fillna(0)

                        session.bulk_insert_mappings(
                            model,
                            chunk.to_dict('records'),
                            render_nulls=True  # Ważne dla MySQL/MariaDB
                        )

                    session.commit()
                    print(f"Zaimportowano {len(df)} rekordów do {table_name}")

        except Exception as e:
            session.rollback()
            print(f"Błąd: {str(e)}")

# Konfiguracja połączeń
databases = {
    'postgresql': (
        f"postgresql://{postgres_config['environment']['POSTGRES_USER']}:"
        f"{postgres_config['environment']['POSTGRES_PASSWORD']}@"
        f"localhost:{postgres_config['ports'][0].split(':')[0]}/"
        f"{postgres_config['environment']['POSTGRES_DB']}"
    ),

    'mariadb': (
        f"mysql+pymysql://"
        f"{docker_config['services']['mariadb']['environment']['MYSQL_USER']}:"
        f"{docker_config['services']['mariadb']['environment']['MYSQL_PASSWORD']}@"
        f"localhost:{docker_config['services']['mariadb']['ports'][0].split(':')[0]}/"
        f"{docker_config['services']['mariadb']['environment']['MYSQL_DATABASE']}"
    )
}
# Przetwarzanie dla każdej bazy danych
for db_name, uri in databases.items():
    print(f"\nProcessing {db_name.upper()}")
    #drop_all_tables(uri)
    SessionFactory = setup_database(uri)  # Pobieramy fabrykę sesji
    import_data(SessionFactory, '10tys')

    # Weryfikacja danych
    try:
        with SessionFactory() as session:  # Nowa sesja z fabryki
            classes = session.query(Class).order_by(Class.id.desc()).limit(10).all()
            print(f"\nPrzykładowe klasy w {db_name}:")
            for c in classes:
                print(f"ID: {c.id}, Nazwa: {c.name}, Nauczyciel ID: {c.teacher_id}")

    except Exception as e:
        print(f"Błąd weryfikacji: {str(e)}")



Processing POSTGRESQL
Błąd: ['teacher_id', 'student_id', 'class_id', 'subject_id']

Przykładowe klasy w postgresql:
ID: 10000, Nazwa: Behavior, Nauczyciel ID: 3964
ID: 9999, Nazwa: Game, Nauczyciel ID: 6910
ID: 9998, Nazwa: Black, Nauczyciel ID: 3319
ID: 9997, Nazwa: Stage, Nauczyciel ID: 2865
ID: 9996, Nazwa: Should, Nauczyciel ID: 4330
ID: 9995, Nazwa: That, Nauczyciel ID: 4863
ID: 9994, Nazwa: Catch, Nauczyciel ID: 9719
ID: 9993, Nazwa: Tv, Nauczyciel ID: 492
ID: 9992, Nazwa: Option, Nauczyciel ID: 3134
ID: 9991, Nazwa: Difference, Nauczyciel ID: 2344

Processing MARIADB
Błąd: ['teacher_id', 'student_id', 'class_id', 'subject_id']

Przykładowe klasy w mariadb:
ID: 10000, Nazwa: Behavior, Nauczyciel ID: 3964
ID: 9999, Nazwa: Game, Nauczyciel ID: 6910
ID: 9998, Nazwa: Black, Nauczyciel ID: 3319
ID: 9997, Nazwa: Stage, Nauczyciel ID: 2865
ID: 9996, Nazwa: Should, Nauczyciel ID: 4330
ID: 9995, Nazwa: That, Nauczyciel ID: 4863
ID: 9994, Nazwa: Catch, Nauczyciel ID: 9719
ID: 9993, Nazwa: